# **NYC Trip Fare Analysis**

Dataset description extracted from [Kaggle](https://www.kaggle.com/datasets/diishasiing/revenue-for-cab-drivers):

- **VendorID**: A unique identifier for the taxi vendor or service provider.
- **tpep_pickup_datetime**: The date and time when the passenger was picked up.
- **tpep_dropoff_datetime**: The date and time when the passenger was dropped off.
- **passenger_count**: The number of passengers in the taxi.
- **trip_distance**: The total distance of the trip in miles or kilometers.
- **RatecodeID**: The rate code assigned to the trip, representing fare types.
- **store_and_fwd_flag**: Indicates whether the trip data was stored locally and then forwarded later (Y/N).
- **PULocationID**: The unique identifier for the pickup location (zone or area).
- **DOLocationID**: The unique identifier for the drop-off location (zone or area).
- **payment_type**: The method of payment used by the passenger (e.g., cash, card).
- **fare_amount**: The base fare for the trip.
- **extra**: Additional charges applied during the trip (e.g., night surcharge).
- **mta_tax**: The tax imposed by the Metropolitan Transportation Authority.
- **tip_amount**: The tip given to the driver, if applicable.
- **tolls_amount**: The total amount of tolls charged during the trip.
- **improvement_surcharge**: A surcharge imposed for the improvement of services.
- **total_amount**: The total fare amount, including all charges and surcharges.
- **congestion_surcharge**: An additional charge for trips taken during high traffic congestion times.

## **1. Set Environment and Import Libraries**

In [1]:
import sys
import os

# Add 'conf' folder to sys.path (if not already present)
conf_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'conf'))
if conf_path not in sys.path:
    sys.path.append(conf_path)

# Now import conf module from conf folder
import conf

# Verifica che i percorsi siano corretti
#print("Main directory:", conf.MAIN_DIR)
#print("Notebook directory:", conf.NOTEBOOK_DIR)
#print("Data directory:", conf.DATA_DIR)

Now, import main packages necessary for the developement of this project:

In [2]:
# Import libraries
import pandas as pd
import datetime
import time
import numpy as np

## **2. Data Importation and Pre-Processing**

In [3]:
# Read data from the .csv file within data folder
dataset = pd.read_csv(f'{conf.DATA_DIR}/data.csv')

/var/folders/w7/0xpq0vds7cz2333831nfq5dc0000gn/T/ipykernel_23463/2694177516.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f'{conf.DATA_DIR}/data.csv')


In [4]:
dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


Print out columns types to check the warning (columns with different data types):

In [5]:
print(dataset.dtypes)

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object


Firstly, create a copy of the original dataset for recovery.

In [6]:
# Create a copy of the original dataset
dataset_old = dataset.copy()

Now, analyze columns' type.

In [7]:
dataset.store_and_fwd_flag.unique()

array(['N', 'Y', nan], dtype=object)

The problem is on 'nan' that are encoded in a wrong way, and also datetime columns must have a specified type.

In [8]:
# Clean up missing values before conversion into type "category"
dataset['store_and_fwd_flag'] = dataset['store_and_fwd_flag'].replace(["", " ", "NaN", "nan"], pd.NA).astype("category")
dataset.store_and_fwd_flag.unique()

['N', 'Y', NaN]
Categories (2, object): ['N', 'Y']

Moreover, dates must be converted into the proper format:

In [9]:
# Convert datetime fields to proper format
dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset['tpep_pickup_datetime'], errors='coerce') # errors = 'coerce' to avoid anomaly
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'], errors='coerce')

In [10]:
print(dataset[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag']].dtypes)

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
store_and_fwd_flag             category
dtype: object


In [11]:
dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


Now, considering how data are described within the Kaggle datacard, try to understand if adjustments are required. It is important that numerical IDs and counters are converted into **integer numbers**. With the format `Int64` it is possible to handle null values for integer data without converting them into `float64`.

Variables which require to be converted into integers are `VendorID`, `passenger_count`, `RatecodeID`, `payment_type`, `DOLocationID` and `PULocationID`.

In [12]:
col_to_modify = ['VendorID', 'passenger_count', 'RatecodeID', 'payment_type', 'DOLocationID', 'PULocationID']

for col in col_to_modify:
    dataset[col] = dataset[col].astype('Int64')

In [13]:
for col in list(dataset_old.columns):
    print('Column :', col, '| before:', dataset_old[col].dtype, ' -> after:', dataset[col].dtype)

Column : VendorID | before: float64  -> after: Int64
Column : tpep_pickup_datetime | before: object  -> after: datetime64[ns]
Column : tpep_dropoff_datetime | before: object  -> after: datetime64[ns]
Column : passenger_count | before: float64  -> after: Int64
Column : trip_distance | before: float64  -> after: float64
Column : RatecodeID | before: float64  -> after: Int64
Column : store_and_fwd_flag | before: object  -> after: category
Column : PULocationID | before: int64  -> after: Int64
Column : DOLocationID | before: int64  -> after: Int64
Column : payment_type | before: float64  -> after: Int64
Column : fare_amount | before: float64  -> after: float64
Column : extra | before: float64  -> after: float64
Column : mta_tax | before: float64  -> after: float64
Column : tip_amount | before: float64  -> after: float64
Column : tolls_amount | before: float64  -> after: float64
Column : improvement_surcharge | before: float64  -> after: float64
Column : total_amount | before: float64  -> a

## **3. Assignements**

### 3.1. Extract all trips with `trip_distance` larger than 50

In [14]:
# Extract all trip with distance > 50
dataset[dataset['trip_distance'] > 50]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
23842,2,2020-01-01 01:53:07,2020-01-01 03:54:41,1,52.30,5,N,262,265,1,300.00,0.00,0.0,61.78,6.12,0.3,370.70,2.5
39013,2,2020-01-01 02:05:07,2020-01-01 03:03:10,1,51.23,5,N,264,264,1,329.00,0.00,0.5,100.78,6.12,0.3,436.70,0.0
41620,1,2020-01-01 03:05:54,2020-01-01 04:16:26,1,53.80,5,N,132,265,1,250.00,0.00,0.0,53.35,16.62,0.3,320.27,0.0
58262,2,2020-01-01 05:36:12,2020-01-01 06:40:06,1,55.23,5,N,132,265,2,170.00,0.00,0.5,0.00,18.26,0.3,189.06,0.0
63024,2,2020-01-01 07:40:30,2020-01-01 08:40:01,1,54.19,5,N,132,265,1,230.00,0.00,0.0,0.00,12.24,0.3,242.54,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6326169,2,2020-01-31 22:47:26,2020-01-31 23:49:14,1,51.83,5,N,132,265,1,220.00,0.00,0.5,48.96,23.99,0.3,293.75,0.0
6331181,2,2020-01-31 23:45:36,2020-02-01 01:00:25,5,57.99,4,N,107,265,1,245.00,0.50,0.5,38.24,6.12,0.3,293.16,2.5
6333801,2,2020-01-31 23:24:16,2020-02-01 01:32:56,1,52.97,4,N,264,265,1,227.00,0.50,0.5,46.16,0.00,0.3,276.96,2.5
6397132,<NA>,2020-01-28 11:54:00,2020-01-28 19:35:00,<NA>,60.36,<NA>,NaN,17,61,<NA>,12.04,0.00,0.5,0.00,12.24,0.3,25.08,0.0


### 3.2 Extract all trips where `payment_type` is missing

In [15]:
# Trips with payment_type missings
dataset[dataset['payment_type'].isna()]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,<NA>,2020-01-01 08:51:00,2020-01-01 09:19:00,<NA>,13.69,<NA>,NaN,136,232,<NA>,51.05,2.75,0.5,0.0,0.00,0.3,54.60,0.0
6339568,<NA>,2020-01-01 08:38:43,2020-01-01 08:51:08,<NA>,3.42,<NA>,NaN,121,9,<NA>,27.06,2.75,0.0,0.0,0.00,0.3,30.11,0.0
6339569,<NA>,2020-01-01 08:27:00,2020-01-01 08:32:00,<NA>,2.20,<NA>,NaN,197,216,<NA>,24.36,2.75,0.5,0.0,0.00,0.3,27.91,0.0
6339570,<NA>,2020-01-01 08:46:00,2020-01-01 08:57:00,<NA>,0.84,<NA>,NaN,262,236,<NA>,26.08,2.75,0.5,0.0,0.00,0.3,29.63,0.0
6339571,<NA>,2020-01-01 08:21:00,2020-01-01 08:38:00,<NA>,7.24,<NA>,NaN,45,142,<NA>,25.28,2.75,0.5,0.0,0.00,0.3,28.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,<NA>,2020-01-31 22:51:00,2020-01-31 23:22:00,<NA>,3.24,<NA>,NaN,237,234,<NA>,17.59,2.75,0.5,0.0,0.00,0.3,21.14,0.0
6405004,<NA>,2020-01-31 22:10:00,2020-01-31 23:26:00,<NA>,22.13,<NA>,NaN,259,45,<NA>,46.67,2.75,0.5,0.0,12.24,0.3,62.46,0.0
6405005,<NA>,2020-01-31 22:50:07,2020-01-31 23:17:57,<NA>,10.51,<NA>,NaN,137,169,<NA>,48.85,2.75,0.0,0.0,0.00,0.3,51.90,0.0
6405006,<NA>,2020-01-31 22:25:53,2020-01-31 22:48:32,<NA>,5.49,<NA>,NaN,50,42,<NA>,27.17,2.75,0.0,0.0,0.00,0.3,30.22,0.0


### 3.3 For each (`PULocationID`, `DOLocationID`) pair, determine the number of trips

Considering that some rows have missing `PULocationID` and/or `DOLocationID`, it doesn't make sense taking into account pairs of locations for which at least one column is missing. Hence, consider non missing values for these fields.

In [16]:
dataset.value_counts(['PULocationID', 'DOLocationID']).reset_index(name='trip_count')

,PULocationID,DOLocationID,trip_count
0,237,236,45539
1,236,236,38775
2,236,237,38264
3,237,237,33909
4,264,264,27928
...,...,...,...
31272,80,108,1
31273,9,70,1
31274,229,6,1
31275,163,245,1


### 3.4 Save all rows with missing `VendorID`, `passenger_count`, `store_and_fwd_flag`, `payment_type` in a new dataframe called `bad`, and remove those rows from the original dataframe

There are two possibilities:
-	`.any(axis=1)`: Removes rows where **at least one** of the specified columns is **NaN**.
-	`.all(axis=1)`: Removes rows where **all specified columns** are **NaN** at the same time.

In this case, the proper solution is `.any(axis=1)`:

In [17]:
bad = dataset[dataset[['VendorID', 'passenger_count', 'payment_type', 'store_and_fwd_flag']].isna().any(axis=1)]
bad

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,<NA>,2020-01-01 08:51:00,2020-01-01 09:19:00,<NA>,13.69,<NA>,NaN,136,232,<NA>,51.05,2.75,0.5,0.0,0.00,0.3,54.60,0.0
6339568,<NA>,2020-01-01 08:38:43,2020-01-01 08:51:08,<NA>,3.42,<NA>,NaN,121,9,<NA>,27.06,2.75,0.0,0.0,0.00,0.3,30.11,0.0
6339569,<NA>,2020-01-01 08:27:00,2020-01-01 08:32:00,<NA>,2.20,<NA>,NaN,197,216,<NA>,24.36,2.75,0.5,0.0,0.00,0.3,27.91,0.0
6339570,<NA>,2020-01-01 08:46:00,2020-01-01 08:57:00,<NA>,0.84,<NA>,NaN,262,236,<NA>,26.08,2.75,0.5,0.0,0.00,0.3,29.63,0.0
6339571,<NA>,2020-01-01 08:21:00,2020-01-01 08:38:00,<NA>,7.24,<NA>,NaN,45,142,<NA>,25.28,2.75,0.5,0.0,0.00,0.3,28.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,<NA>,2020-01-31 22:51:00,2020-01-31 23:22:00,<NA>,3.24,<NA>,NaN,237,234,<NA>,17.59,2.75,0.5,0.0,0.00,0.3,21.14,0.0
6405004,<NA>,2020-01-31 22:10:00,2020-01-31 23:26:00,<NA>,22.13,<NA>,NaN,259,45,<NA>,46.67,2.75,0.5,0.0,12.24,0.3,62.46,0.0
6405005,<NA>,2020-01-31 22:50:07,2020-01-31 23:17:57,<NA>,10.51,<NA>,NaN,137,169,<NA>,48.85,2.75,0.0,0.0,0.00,0.3,51.90,0.0
6405006,<NA>,2020-01-31 22:25:53,2020-01-31 22:48:32,<NA>,5.49,<NA>,NaN,50,42,<NA>,27.17,2.75,0.0,0.0,0.00,0.3,30.22,0.0


### 3.5 Add a duration column storing how long each trip has taken (use `tpep_pickup_datetime`, `tpep_dropoff_datetime`)

Since `tpep_pickup_datetime` are the date and time when the passenger was picked up and `tpep_dropoff_datetime` are the date and time when the passenger was dropped off, the duration can be simply computed as this difference: `tpep_dropoff_datetime` -  `tpep_pickup_datetime`.

In [18]:
dataset['trip_duration'] = dataset['tpep_dropoff_datetime'] - dataset['tpep_pickup_datetime']
dataset[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_duration']]

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_duration
0,2020-01-01 00:28:15,2020-01-01 00:33:03,0 days 00:04:48
1,2020-01-01 00:35:39,2020-01-01 00:43:04,0 days 00:07:25
2,2020-01-01 00:47:41,2020-01-01 00:53:52,0 days 00:06:11
3,2020-01-01 00:55:23,2020-01-01 01:00:14,0 days 00:04:51
4,2020-01-01 00:01:58,2020-01-01 00:04:16,0 days 00:02:18
...,...,...,...
6405003,2020-01-31 22:51:00,2020-01-31 23:22:00,0 days 00:31:00
6405004,2020-01-31 22:10:00,2020-01-31 23:26:00,0 days 01:16:00
6405005,2020-01-31 22:50:07,2020-01-31 23:17:57,0 days 00:27:50
6405006,2020-01-31 22:25:53,2020-01-31 22:48:32,0 days 00:22:39


The result is a `Timedelta` column, which represents the time difference as number of days, hours, minutes and seconds.

### 3.6 For each pickup location, determine how many trips have started there 

The pickup location is stored within `PULocationID` column. It is simply necessary to compute values distribution of this column:

In [19]:
dataset.PULocationID.value_counts().reset_index(name='trip_count')

,PULocationID,trip_count
0,237,292989
1,161,282213
2,236,272592
3,162,235602
4,186,228746
...,...,...
256,245,2
257,176,2
258,30,1
259,172,1


### 3.7 Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)

In [20]:
# Generate time intervals (30 min bins)
bins = pd.date_range(start='00:00:00', end='23:59:59', freq='30min').time
# Include also the interval "23:30 - 23:59"
bins = np.append(bins, datetime.time(23, 59, 59))

# Define labels (e.g., "02:00-02:30", ...)
labels = [f"{bins[i].strftime('%H:%M')} - {bins[i+1].strftime('%H:%M')}" for i in range(len(bins)-1)]
labels[-1] = '23:30 - 00:00'

In [21]:
dataset['pickup_time_interval'] = pd.cut(dataset['tpep_pickup_datetime'].dt.time, bins=bins, labels=labels, include_lowest=True)

In [22]:
# Check the presence of null values within pickup_time_interval
dataset[dataset['pickup_time_interval'].isna()]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration,pickup_time_interval


In [23]:
dataset

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration,pickup_time_interval
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.20,1,N,238,239,1,6.00,3.00,0.5,1.47,0.00,0.3,11.27,2.5,0 days 00:04:48,00:00 - 00:30
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.20,1,N,239,238,1,7.00,3.00,0.5,1.50,0.00,0.3,12.30,2.5,0 days 00:07:25,00:30 - 01:00
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.60,1,N,238,238,1,6.00,3.00,0.5,1.00,0.00,0.3,10.80,2.5,0 days 00:06:11,00:30 - 01:00
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.80,1,N,238,151,1,5.50,0.50,0.5,1.36,0.00,0.3,8.16,0.0,0 days 00:04:51,00:30 - 01:00
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0.00,1,N,193,193,2,3.50,0.50,0.5,0.00,0.00,0.3,4.80,0.0,0 days 00:02:18,00:00 - 00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,<NA>,2020-01-31 22:51:00,2020-01-31 23:22:00,<NA>,3.24,<NA>,NaN,237,234,<NA>,17.59,2.75,0.5,0.00,0.00,0.3,21.14,0.0,0 days 00:31:00,22:30 - 23:00
6405004,<NA>,2020-01-31 22:10:00,2020-01-31 23:26:00,<NA>,22.13,<NA>,NaN,259,45,<NA>,46.67,2.75,0.5,0.00,12.24,0.3,62.46,0.0,0 days 01:16:00,22:00 - 22:30
6405005,<NA>,2020-01-31 22:50:07,2020-01-31 23:17:57,<NA>,10.51,<NA>,NaN,137,169,<NA>,48.85,2.75,0.0,0.00,0.00,0.3,51.90,0.0,0 days 00:27:50,22:30 - 23:00
6405006,<NA>,2020-01-31 22:25:53,2020-01-31 22:48:32,<NA>,5.49,<NA>,NaN,50,42,<NA>,27.17,2.75,0.0,0.00,0.00,0.3,30.22,0.0,0 days 00:22:39,22:00 - 22:30


### 3.8 For each interval, determine the average number of passengers and the average fare amount

In [24]:
dataset.groupby('pickup_time_interval', observed=False).agg(avg_passenger_count=('passenger_count', 'mean'),
                                                            avg_fare_amount=('fare_amount', 'mean')).reset_index()

,pickup_time_interval,avg_passenger_count,avg_fare_amount
0,00:00 - 00:30,1.572854,13.682459
1,00:30 - 01:00,1.584273,13.302491
2,01:00 - 01:30,1.578807,12.766396
3,01:30 - 02:00,1.589424,12.332639
4,02:00 - 02:30,1.587403,12.159536
5,02:30 - 03:00,1.587833,12.129466
6,03:00 - 03:30,1.581925,12.608487
7,03:30 - 04:00,1.585897,13.279612
8,04:00 - 04:30,1.580107,14.585496
9,04:30 - 05:00,1.516206,17.115791


### 3.9 For each payment type and each interval, determine the average fare amount

In [25]:
# Compute the average fare amount for each (payment_type, pickup_time_interval)
avg_fare = dataset.groupby(['payment_type', 'pickup_time_interval'], observed=False).agg(avg_fare_amount=('fare_amount', 'mean')).reset_index()
avg_fare

,payment_type,pickup_time_interval,avg_fare_amount
0,1,00:00 - 00:30,13.868641
1,1,00:30 - 01:00,13.472738
2,1,01:00 - 01:30,12.822628
3,1,01:30 - 02:00,12.358248
4,1,02:00 - 02:30,12.008941
...,...,...,...
235,5,21:30 - 22:00,NaN
236,5,22:00 - 22:30,NaN
237,5,22:30 - 23:00,NaN
238,5,23:00 - 23:30,NaN


### 3.10 For each payment type, determine the interval when the average fare amount is maximum

In [29]:
# Find the interval with the maximum average fare for each payment type
avg_fare.loc[avg_fare.groupby('payment_type', observed=False)['avg_fare_amount'].idxmax()]

,payment_type,pickup_time_interval,avg_fare_amount
10,1,05:00 - 05:30,21.260986
58,2,05:00 - 05:30,14.856701
110,3,07:00 - 07:30,10.950938
154,4,05:00 - 05:30,6.634043
227,5,17:30 - 18:00,0.000000


### 3.11 For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum

In [27]:
dataset['ratio_tip_fare_amounts'] = dataset['tip_amount'] / dataset['fare_amount']
dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration,pickup_time_interval,ratio_tip_fare_amounts
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.2,1,N,238,239,1,...,3.0,0.5,1.47,0.0,0.3,11.27,2.5,0 days 00:04:48,00:00 - 00:30,0.245000
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.2,1,N,239,238,1,...,3.0,0.5,1.50,0.0,0.3,12.30,2.5,0 days 00:07:25,00:30 - 01:00,0.214286
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.6,1,N,238,238,1,...,3.0,0.5,1.00,0.0,0.3,10.80,2.5,0 days 00:06:11,00:30 - 01:00,0.166667
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.8,1,N,238,151,1,...,0.5,0.5,1.36,0.0,0.3,8.16,0.0,0 days 00:04:51,00:30 - 01:00,0.247273
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0.0,1,N,193,193,2,...,0.5,0.5,0.00,0.0,0.3,4.80,0.0,0 days 00:02:18,00:00 - 00:30,0.000000


In [28]:
# Now, group by payment_type and pickup_time_interval, and for each payment type get the interval where the ratio is maximum
x = dataset[['payment_type', 'pickup_time_interval', 'ratio_tip_fare_amounts']]
x.loc[x.groupby('payment_type')['ratio_tip_fare_amounts'].idxmax()]

KeyError: '[nan] not in index'

### 3.12 Find the location with the highest average fare amount

### 3.13 Build a new dataframe (called `common`) where, for each pickup location we keep all trips to the 5 most common destinations (i.e. each pickup location can have different common destinations)

### 3.14 On the `common` dataframe, for each payment type and each interval, determine the average fare amount

### 3.15 Compute the difference of the average fare amount computed in the previous point with those computed at point 9

### 3.16 Compute the ratio between the differences computed in the previous point and those computed in point 9. Note: you have to compute a ratio for each pair (payment type, interval)

### 3.17 Build chains of trips. Two trips are consecutive in a chain if (a) they have the same VendorID, (b) the pickup location of the second trip is also the dropoff location of the first trip, (c) the pickup time of the second trip is after the dropoff time of the first trip, and (d) the pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip.

**Hint**: Add a column `chain` to the dataset. A chain can have more than two trips.